In [1]:
import pickle
import pandas as pd
import nltk
import re

In [2]:
model = pickle.load(open('./model.pkl', 'rb'))
countvect = pickle.load(open('./countvect.pkl', 'rb'))

/home/rohit/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LogisticRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/rohit/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator CountVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
from nltk.corpus import stopwords

In [4]:
def clean_texts(texts):
    stwords = stopwords.words('english')
    l = len(texts)/10
    temp_texts = []
    for i in range(len(texts)):
        text = re.sub('\d','0',texts[i])
        if 'www.' in text or 'http:' in text or 'https:' in text or '.com' in text: # remove links and urls
            text = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", " ", text)
        
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = text.split()
        text = [word for word in text if not word in stwords] # remove stopwords 
        text = ' '.join(text)
        temp_texts.append(text)
        if i%l==0:
            print('--'+str(int(i/l)*10)+'%', end='')
    print('--100%--Done !')
    return temp_texts

In [15]:
from scrapy import AmazonReviews

In [16]:
obj = AmazonReviews("https://www.amazon.in/LG-Inverter-Frost-Free-Refrigerator-GL-I292RPZX/dp/B08X72GY5Q/ref=lp_41909197031_1_1")

        LG 260L 3 Star Smart Inverter Frost-Free Double Door Refrigerator (GL-I292RPZX, Shiny Steel, Door Cooling+)       


In [17]:
reviews = obj.scrap(pages=2)

In [18]:
print(reviews)

['Good for a 5 members family', 'LG was one of the trusted brand in electronic items, but this model is worst. Within 5 months it shows cooling issues. Only freezer is cooling up lower fridge storage not cooling, full of water drops falling from freezer bottom size.', "                    The media could not be loaded.                \xa0I received the product on time and LG demo was done on the same day of deilvery.PROS:-Fridge build and finish is good.-There is lot of space for storage.-It runs with smart inventor technology-has huge fruit and vegetables tray.-DOOR COOLING PLUS IS GREAT FEATURES AND USEFUL, WHICH GIVE COOLING FROM FRONT AND BACK.-Smart diagnosis (i haven't use this feature, but it useful to diagnosis any issue)-So overall this is excellent buy for my requirements and I can recommend this product.Few cons which I observed:-There is no light freezer compartment.-No separate tray or space for dairy productsThis 2 cons were not dealer break for me", "Over all it's good",

In [9]:
reviews = clean_texts(reviews)
print(reviews)

--0%--100%--Done !
['good members family', 'lg one trusted brand electronic items model worst within months shows cooling issues freezer cooling lower fridge storage cooling full water drops falling freezer bottom size', 'media could loaded received product time lg demo done day deilvery pros fridge build finish good lot space storage runs smart inventor technology huge fruit vegetables tray door cooling plus great features useful give cooling front back smart diagnosis use feature useful diagnosis issue overall excellent buy requirements recommend product cons observed light freezer compartment separate tray space dairy productsthis cons dealer break', 'good', 'nice product timely delivery', 'side looking superb inside plastic material delicate compare old lg refrigerator overall experience ok', 'good product delivered', 'nice product good quality', 'good', 'lg product always good', 'good', 'bad post sell service', 'good product', 'fast cooling low noise feature useful', 'every thing 

In [10]:
reviews_vect = countvect.transform(reviews)

In [11]:
pred = model.predict(reviews_vect)

In [14]:
pd.DataFrame({'Reviews':reviews,'Sentiment':pred})

,Reviews,Sentiment
0,Good for a 5 members family,1
1,LG was one of the trusted brand in electronic ...,0
2,The media could not be loa...,1
3,Over all it's good,1
4,Nice product timely delivery,1
5,OUT SIDE LOOKING SUPERB......BUT INSIDE PLASTI...,1
6,Good product delivered,1
7,Nice product and good quality,1
8,Good,1
9,LG product are always good,1


In [40]:
from sklearn.metrics import accuracy_score, confusion_matrix

accuracy_score(pred, labels)

0.9052

In [41]:
confusion_matrix(pred, df['Labels'])

array([[4609,  460],
       [ 488, 4443]])

In [42]:
df['Reviews'][12]

'great read: i thought this book was brilliant, but yet realistic. it showed me that to error is human. i loved the fact that this writer showed the loving side of god and not the revengeful side of him. i loved how it twisted and turned and i could not put it down. i also loved the glass castle.'

In [43]:
df['Labels'][12]

1

In [45]:
df['Reviews'][0]

'stuning even for the non-gamer: this sound track was beautiful! it paints the senery in your mind so well i would recomend it even to people who hate vid. game music! i have played the game chrono cross but out of all of the games i have ever played it has the best music! it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. it would impress anyone who cares to listen! ^_^'

In [46]:
from bs4 import BeautifulSoup as bs
import requests

In [48]:
link1 = "https://www.amazon.in/OnePlus-Nord-Bahamas-128GB-Storage/dp/B09RG5R5FG/ref=sr_1_3?crid=OS3JBJSOS7MW&keywords=oneplus+nord+ce+2+5g&qid=1665410198&qu=eyJxc2MiOiIzLjIzIiwicXNhIjoiMi43MyIsInFzcCI6IjIuNDAifQ%3D%3D&sprefix=one%2Caps%2C259&sr=8-3"
print(link1)

https://www.amazon.in/OnePlus-Nord-Bahamas-128GB-Storage/dp/B09RG5R5FG/ref=sr_1_3?crid=OS3JBJSOS7MW&keywords=oneplus+nord+ce+2+5g&qid=1665410198&qu=eyJxc2MiOiIzLjIzIiwicXNhIjoiMi43MyIsInFzcCI6IjIuNDAifQ%3D%3D&sprefix=one%2Caps%2C259&sr=8-3


In [21]:
pred = [0,1,1,1,1,0,1,0,0]

In [26]:
round((30/100)*100)

30

In [27]:
df = pd.read_csv('reviews.csv')

In [29]:
df.to_html('reviews.html')